<a href="https://colab.research.google.com/github/R12942159/NTU_ML/blob/Hw3/hw3_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Module

In [ ]:
import csv
import time
import sys
import os
import random

# other library
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# PyTorch library
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data
from tqdm.notebook import tqdm

# Fix random seed

In [ ]:
SEED = 5566 # Do not modify
use_gpu = torch.cuda.is_available()
device = torch.device("cuda" if use_gpu else "cpu")

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
random.seed(SEED)
np.random.seed(SEED)

# Dataset Download:

In [ ]:
!gdown --id "1Rlt_mU-siC6AayAjNMXZ8N8fURWg5iLB" --output "trainX.npy"
!gdown --id "1v7i5rP2UQv_2ju8akh69M5VzEjRJV3tB" --output "visualization_X.npy"


/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1Rlt_mU-siC6AayAjNMXZ8N8fURWg5iLB
To: /content/trainX.npy
100% 221M/221M [00:05<00:00, 40.9MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1v7i5rP2UQv_2ju8akh69M5VzEjRJV3tB
To: /content/visualization_X.npy
100% 15.4M/15.4M [00:00<00:00, 21.7MB/s]


# Set Hyper-parameters

In [ ]:
#TODO: Modified the hyper-parameter
NUM_EPOCH = 5
BATCH_SIZE = 32
LATENT_DIM = 32
REDUCED_DIM = 8
NUM_ITER = 1000
REDUCED_METHOD = 'pca' # or 'tsne'
lr = 5e-4

In [ ]:
MODEL_NAME = 'model.pth'
DATA_PATH = 'trainX.npy'

# Define Dataset

In [ ]:
class Dataset(data.Dataset):
    def __init__(self, data_path):
        self.total_img = torch.from_numpy(np.load(data_path)).float()
        self.total_img = self.total_img.permute(0, 3, 1, 2)
        self.total_img = self.total_img/255

    def normalize(self, img):
        # TODO: normalize the dataset
        return img

    def augment(self, img):
        # TODO: do augmentation while loading image
        return img

    def __len__(self):
        return len(self.total_img)

    def __getitem__(self, index):
        img = self.total_img[index]
        img_aug = self.augment(img)

        img_aug = self.normalize(img_aug)
        img = self.normalize(img)
        return img_aug, img

# Define Model Architerchure

**Please finish this block to run this code!**


In [ ]:
class Net(nn.Module):
    def __init__(self, image_channels=3, latent_dim=128):
        super(Net, self).__init__()
        self.latent_dim = latent_dim
        self.img_size = 32

        self.encoder = nn.Sequential(
            nn.Conv2d(image_channels, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            # TODO: define your own structure
        )

        # TODO: check the dimension if you modified the structure
        self.fc1 = nn.Linear(N, self.latent_dim)

        # TODO: check the dimension if you modified the structure
        self.fc2 = nn.Linear(self.latent_dim, N)

        self.decoder = nn.Sequential(
           # TODO: define yout own structure
           # Hint: nn.ConvTranspose2d(...)
        )

    def forward(self, x):

        feature_map = self.encoder(x)
        latent_vec = self.fc1(feature_map.reshape(feature_map.shape[0], -1))
        feature_map2 = self.fc2(latent_vec)
        x_res = self.decoder(feature_map2)

        return latent_vec, x_res

# Define Training Process

In [ ]:
def training(train, val, model, device, n_epoch, batch, save_name, lr):
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print('=== start training, parameter total:%d, trainable:%d' % (total, trainable))
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr,
                             weight_decay=1e-5)
    best_loss = 100
    for epoch in range(n_epoch):
        total_loss = 0

        # training set
        model.train()
        idx = 0
        for image_aug, image in tqdm(train):
            image = image.to(device, dtype=torch.float)
            image_aug = image_aug.to(device, dtype=torch.float)
            _, reconsturct = model(image_aug)
            loss = criterion(reconsturct, image)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += (loss.item() / len(train))

            print('[Epoch %d | %d/%d] loss: %.4f' %
                 ((epoch+1), idx*batch, len(train)*batch, loss.item()), end='\r')
            idx += 1
        print("\n  Training  | Loss:%.4f " % total_loss)

        # validation set
        model.eval()
        total_loss = 0
        idx = 0
        with torch.no_grad():
            for image_aug, image in tqdm(val):
                image = image.to(device, dtype=torch.float)
                image_aug = image_aug.to(device, dtype=torch.float)
                _, reconstruct = model(image_aug)

                loss = criterion(reconstruct, image)
                total_loss += (loss.item() / len(val))
                idx += 1
            print(" Validation | Loss:%.4f " % total_loss)
        # save model
        if total_loss < best_loss:
                best_loss = total_loss
                print("saving model with loss %.4f...\n" % total_loss)
                torch.save(model.state_dict(), "%s" % save_name)

# Define Clustering Process

In [ ]:
def clustering(model, device, loader, n_iter, reduced_method, reduced_dim, perplexity):
    assert reduced_method in ['pca', 'tsne', None]

    model.eval()
    latent_vec = torch.tensor([]).to(device, dtype=torch.float)
    for idx, (image_aug, image) in enumerate(loader):
        print("predict %d / %d" % (idx, len(loader)) , end='\r')
        image = image.to(device, dtype=torch.float)
        latent, r = model(image)
        latent_vec = torch.cat((latent_vec, latent), dim=0)

    latent_vec = latent_vec.cpu().detach().numpy()

    if reduced_method == 'tsne':
        tsne = TSNE(n_components=reduced_dim, verbose=1, method='exact', perplexity=perplexity, n_iter=n_iter)
        latent_vec = tsne.fit_transform(latent_vec)
    elif reduced_method == 'pca':
        pca = PCA(n_components=reduced_dim, copy=False, whiten=True, svd_solver='full')
        latent_vec = pca.fit_transform(latent_vec)

    kmeans = KMeans(n_clusters=2, random_state=0, max_iter=n_iter).fit(latent_vec)
    return kmeans.labels_

# Define write function

In [ ]:
def write_output(predict_result, file_name='predict.csv'):
    with open(file_name, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['id', 'label'])
        for i in range(len(predict_result)):
            writer.writerow([str(i), str(predict_result[i])])

# Main Process

In [ ]:
# build dataset
dataset = Dataset(DATA_PATH)
print(len(dataset))

# Random split
train_set_size = int(len(dataset) * 0.85)
valid_set_size = len(dataset) - train_set_size
train_set, valid_set = data.random_split(dataset, [train_set_size, valid_set_size])

# set data loader
train_loader = data.DataLoader(train_set, batch_size=BATCH_SIZE, num_workers=1, shuffle=True)
valid_loader = data.DataLoader(valid_set, batch_size=BATCH_SIZE, num_workers=1, shuffle=False)

model = Net(latent_dim=LATENT_DIM).to(device)
print(model)
training(train_loader, valid_loader, model, device, NUM_EPOCH, BATCH_SIZE, MODEL_NAME, lr)



# Inference

In [ ]:
test_loader = data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False)
model.load_state_dict(torch.load(MODEL_NAME))
predicted = clustering(model, device, test_loader, NUM_ITER, reduced_method=REDUCED_METHOD, reduced_dim=REDUCED_DIM, perplexity=15)
write_output(predicted, 'pred.csv')